#  Setup et Imports

In [50]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from notebook_helpers import load_all_data, logger

# Chargement des Données

In [23]:
df_anime, df_genres, df_studios = load_all_data()
print(df_anime.shape)
print(df_genres.shape)
print(df_studios.shape)

2025-11-09 21:26:58 - INFO - 📚 Animes chargés : 20000 lignes, 8 colonnes
2025-11-09 21:26:58 - INFO - 🏷️ Genres chargés : 42152 lignes
2025-11-09 21:26:58 - INFO - 🎬 Studios chargés : 12979 lignes


(20000, 8)
(42152, 3)
(12979, 3)


In [10]:
sum_genres = df_genres['genre'].nunique()
print(sum_genres)

19


# Matrice de contingence Anime x Genres

In [30]:
# 1. Créer la matrice anime x genre
df_merged = pd.merge(df_anime, df_genres[['anime_id', 'genre']], on='anime_id')
anime_genre_matrix = pd.pivot_table(
    df_merged,
    index = 'title',
    columns = 'genre',
    values = 'anime_id',
    aggfunc = 'count',
    fill_value = 0
)
print(anime_genre_matrix.shape)

(17781, 19)


# Cosine Similarity entre anime

In [31]:
similarity_matrix = cosine_similarity(anime_genre_matrix)
print(similarity_matrix.shape)


(17781, 17781)


/Users/yassermourabih/Projet/anime-data-platform/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/yassermourabih/Projet/anime-data-platform/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/yassermourabih/Projet/anime-data-platform/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [49]:
print (similarity_matrix[0])  # Similarité avec le premier anime

[1.         0.40824829 0.         ... 0.         0.         0.        ]


# Fonction de recommandation

In [ ]:
def get_recommendations(anime_title, similarity_matrix, top_k=5):
    """
    Recommande les K animes les plus similaires
    
    Args:
        anime_title (str): Titre de l'anime de référence
        similarity_matrix (np.ndarray): Matrice de similarité
        top_k (int): Nombre de recommandations
        
    Returns:
        pd.Series: Top K animes similaires avec leurs scores
    """
    # Vérifier que l'anime existe
    anime_genre_matrix_index = anime_genre_matrix.index
    if anime_title not in anime_genre_matrix_index:
        logger.warning(f"Anime '{anime_title}' not found in the dataset.")
        return None

    # Trouver la position de l'anime dans la matrice
    index = anime_genre_matrix_index.get_loc(anime_title)

    # Récupérer les scores de similarité
    sim_scores = pd.Series(
        similarity_matrix[index], 
        index=anime_genre_matrix_index
    )

    # Trier par ordre décroissant
    sim_scores = sim_scores.sort_values(ascending=False)

    # Exclure l'anime lui-même
    sim_scores = sim_scores.drop(anime_title)
    
    # Filtrage anti-doublons (séquelles, OVA, etc.)
    final_recommendations = []
    seen_franchises = set()
    source_root = anime_title[:10].lower()
    seen_franchises.add(source_root)

    for title, score in sim_scores.items():
        candidate_root = title[:10].lower()

        if candidate_root in seen_franchises:   
            continue

        final_recommendations.append((title, score))
        seen_franchises.add(candidate_root)
        if len(final_recommendations) >= top_k:
            break
    
    # Convertir en Series
    if final_recommendations:
        titles, scores = zip(*final_recommendations)
        logger.info(f"Extracted {len(titles)} recommendations for '{anime_title}'.")
        return pd.Series(scores, index=titles)
    else:
        logger.warning(f"No recommendations found for '{anime_title}'.")
        return pd.Series(dtype=float)

In [62]:
# Test de la fonction de recommandation

# 1. Tester avec un anime aléatoire
test_anime = 'Shingeki no Kyojin'    #anime_genre_matrix.index[100]  # Prendre le 100ème anime
print(f"🎯 Recommandations pour '{test_anime}' :")
print("-" * 60)
recs = get_recommendations(test_anime, similarity_matrix, top_k=10)
print(recs)
test_anime = 'Naruto'    #anime_genre_matrix.index[100]  # Prendre le 100ème anime
print(f"🎯 Recommandations pour '{test_anime}' :")
print("-" * 60)
recs = get_recommendations(test_anime, similarity_matrix, top_k=10)
print(recs)





2025-11-09 23:09:02 - INFO - Extracted 10 recommendations for 'Shingeki no Kyojin'.
2025-11-09 23:09:02 - WARNING - Anime 'Naruto' not found in the dataset.


🎯 Recommandations pour 'Shingeki no Kyojin' :
------------------------------------------------------------
Vanitas no Carte                                0.894427
Tokyo Majin Gakuen Kenpucho: Tou Dai Ni Maku    0.894427
Hikari no Ou 2nd Season                         0.894427
Vanitas no Carte Part 2                         0.894427
Hikari no Ou                                    0.894427
LOVELESS                                        0.894427
Tsubasa: Shunraiki                              0.894427
Owari no Seraph                                 0.894427
Houseki no Kuni                                 0.894427
Katsute Kami Datta Kemono-tachi e               0.894427
dtype: float64
🎯 Recommandations pour 'Naruto' :
------------------------------------------------------------
None
